# Apache Spark Case Study
##### by Esaq

In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, max, min, sum, to_date, regexp_replace

spark = SparkSession.builder.appName('BankApp').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/05 17:14:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Loading the CSV files as Dataframes

In [5]:
loan_df = spark.read.csv("loan.csv", header=True, inferSchema=True)
credit_df = spark.read.csv('credit card.csv', header=True, inferSchema=True)
txn_df = spark.read.csv("txn.csv", header= True, inferSchema=True)

## Loan CSV
### No. of Loans in Each Category

In [6]:
loan_df.groupBy("Loan Category").count().show()

+------------------+-----+
|     Loan Category|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



### Cleaning the commas in the numbers
### No. of People who have taken loan more than 100000

In [4]:
loan_df = (loan_df
        .withColumn("Loan Amount", regexp_replace(col("Loan Amount").cast("string"), ",", ""))
        .withColumn("Loan Amount", col("Loan Amount").cast("int"))
)

loan_df.filter(col("Loan Amount")>100000).count()

450

### No. of people with income greater than 60000

In [5]:
loan_df.filter(col("Income")>60000).count()

198

### Number of people with 2 or more returned cheques and income less than 50000

In [ ]:
loan_df.filter((col(" Returned Cheque")>=2) & (col("Income")<50000)).count()

137

### Number of people with 2 or more returned cheques and are single


In [ ]:
loan_df.filter((col("Marital Status")=="SINGLE") & (col(" Returned Cheque")>=2)).count()

111

### Number of people with expenditure over 50000 a month

In [ ]:
loan_df.filter(col("Expenditure")>50000).count()

6

## Credit Card CSV

### Credit card users in Spain

In [ ]:
credit_df.filter(col("Geography")=="Spain").count()

2477

## Transaction CSV

### Maximum withdrawal amount in transactions


In [ ]:
txn_df.select(max(' WITHDRAWAL AMT ')).show()

+---------------------+
|max( WITHDRAWAL AMT )|
+---------------------+
|        4.594475464E8|
+---------------------+



### MINIMUM WITHDRAWAL AMOUNT OF AN ACCOUNT in txn.csv


In [ ]:
txn_df.select(min('WITHDRAWAL AMT')).show()

+-------------------+
|min(WITHDRAWAL AMT)|
+-------------------+
|               0.01|
+-------------------+



### MAXIMUM DEPOSIT AMOUNT OF AN ACCOUNT


In [ ]:
txn_df.select(max('DEPOSIT AMT')).show()

+----------------+
|max(DEPOSIT AMT)|
+----------------+
|         5.448E8|
+----------------+



### MINIMUM DEPOSIT AMOUNT OF AN ACCOUNT


In [ ]:
txn_df.select(min('DEPOSIT AMT')).show()

+----------------+
|min(DEPOSIT AMT)|
+----------------+
|            0.01|
+----------------+



### Sum of balance in every bank account


In [ ]:
txn_df.groupBy("Account No").agg(sum("BALANCE AMT").alias("Sum_Balance")).show()

+-------------+--------------------+
|   Account No|         Sum_Balance|
+-------------+--------------------+
|409000438611'|-2.49486577068339...|
|     1196711'|-1.60476498101275E13|
|     1196428'| -8.1418498130721E13|
|409000493210'|-3.27584952132095...|
|409000611074'|       1.615533622E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000493201'|1.0420831829499985E9|
|409000438620'|-7.12291867951358...|
|409000362497'| -5.2860004792808E13|
+-------------+--------------------+



### Number of transaction on each date


In [ ]:
txn_df.groupBy("VALUE DATE").count().orderBy("VALUE DATE").show()

+----------+-----+
|VALUE DATE|count|
+----------+-----+
|  1-Apr-17|    1|
|  1-Aug-15|   75|
|  1-Aug-16|   85|
|  1-Aug-17|   65|
|  1-Aug-18|  144|
|  1-Dec-15|   96|
|  1-Dec-16|  106|
|  1-Dec-17|   45|
|  1-Dec-18|   97|
|  1-Feb-16|   97|
|  1-Feb-17|   81|
|  1-Feb-18|   87|
|  1-Feb-19|   79|
|  1-Jan-15|    3|
|  1-Jan-16|   59|
|  1-Jan-18|   53|
|  1-Jan-19|   57|
|  1-Jul-15|   25|
|  1-Jul-16|  111|
|  1-Jul-17|  243|
+----------+-----+
only showing top 20 rows


### List of customers with withdrawal amount more than 1 lakh


In [ ]:
txn_df.filter(col(" WITHDRAWAL AMT ")>100000).select("Account No", " WITHDRAWAL AMT ", "VALUE DATE").show()

+-------------+----------------+----------+
|   Account No| WITHDRAWAL AMT |VALUE DATE|
+-------------+----------------+----------+
|409000611074'|        133900.0| 16-Aug-17|
|409000611074'|        195800.0| 16-Aug-17|
|409000611074'|        143800.0| 16-Aug-17|
|409000611074'|        331650.0| 16-Aug-17|
|409000611074'|        129000.0| 16-Aug-17|
|409000611074'|        230013.0| 16-Aug-17|
|409000611074'|        367900.0| 16-Aug-17|
|409000611074'|        108000.0| 16-Aug-17|
|409000611074'|        141000.0| 16-Aug-17|
|409000611074'|        206000.0| 16-Aug-17|
|409000611074'|        242300.0|  6-Sep-17|
|409000611074'|        113250.0|  6-Sep-17|
|409000611074'|        206900.0|  6-Sep-17|
|409000611074'|        276000.0|  6-Sep-17|
|409000611074'|        171000.0|  6-Sep-17|
|409000611074'|        189800.0|  6-Sep-17|
|409000611074'|        271323.0|  6-Sep-17|
|409000611074'|        200600.0|  6-Sep-17|
|409000611074'|        176900.0|  6-Sep-17|
|409000611074'|        150050.0|